## Chapter 2: Sets and Maps

### Hashing: 


Magic functions are invoked automatically when an object of that class is used.

In [1]:
hash(3)

3

In [2]:
hash("cat")

5688788956667111595

In [3]:
hash("cat") % 10 # 10 is the number of elements in list

5

In [6]:
hash("dog")

-2325843229467157243

### Linear probing
Hash values are not unique, there can be collisions.
Avoiding collision - linear probing - if collision, advance to next empty location.

After linear probing, we might have to look through next index if there was collision

Adding: hashing, collision, load factor...

### HashSet implementation

#### Class notes

In [17]:
class HashSet:
    def __init__(self, contents = []): # a = HashSet() - __init__ will be invoked
        self.items = [None] * 10 # we'll start with list of arbitary 10 elements - it can be changed later based on load factor
        self.numItems = 0 # initially 0, it'll increase as we add elements

        for item in contents:
            self.add(item) 


    def __add(item, items): #helper/private function - this add function will be a private/intenral function - won't be used outside of the class
        idx = hash(item) % len(items) # get a non-ridiculous hash > now we'll have to detect collision and do linear probing

        loc = -1 # check for collision case, not a storage location

        # in the while loop we'll update location by linear probing
        while items[idx] != None: #initially it'll be None, if there is already data there we'll have to do linear probing
            if items[idx] == item: #if it's duplicate, discard
                return False
            
            if (loc < 0) and (type(items[idx]) == HashSet.__Placeholder):
                loc = idx

            idx = (idx + 1) % len(items) # continue the while loop
        
        if loc < 0:
            loc = idx

        items[loc] = item

        return True
    
    # in order to keep constant time, we need to keep quarter of the list empty. if the length of list changes, the indeces will change, and we'll have to rehash the indices

    def __rehash(oldList, newList): # we'll take the old and new list
        for x in oldList: # for every elements in our old list
            if (x != None) and (type(x) != HashSet.__Placeholder):
                HashSet.__add(x, newList)
        return newList
    
    def add(self, item):
        if HashSet.__add(item, self.items): # check if an unique element was added to the lsit. If it returns false, we just move on, otherwise we check load factor
            self.numItems += 1
            load = self.numItems / len(self.items)
            if load >= 0.75:
                self.items = HashSet.__rehash(self.items, [None]*2*len(self.items)) # returns new empty list twice the length of old list

    # Removing or discarding elements - we can add None before a None, but not in the middle of a chain - and add placeholder instead
    # iterate over items to find value, check if it's the next element is None, replace with None or Placeholder


    class __Placeholder:
        def __init__(self): # defining constructors for objects
            pass

        def __eq__(self, other):
            return False
        
    def __remove(item, items): # since it's a private function, we won't have to call self
        # where is the item located - hashing and linear probing
        idx = hash(item) % len(items)

        while items[idx] != None: # if items under that hash is not None
            if items[idx] == item: # if item equals to  item
                nextIdx = (idx + 1) % len(items)

                if items[nextIdx] == None: # if the next element is None, then we're at the end of the list
                    items[idx] = None # set to None
                else:
                    items[idx] = HashSet.__Placeholder()
                return True
            idx = (idx+1) %len(items)
        return False
    
    def remove(self, item):
        if HashSet.__remove(item, self.items): # check if we have removed item from the list self.items is True
            self.numItems -= 1

            load = max(self.numItems, 10) / len(self.items) # we'll not shrink list beyond 10
            
            if load <= 0.25:
                self.items = HashSet.__rehash(self.items, [None] * len(self.items)//2)

        else: # __remove returned False as the value is not in the list
            raise KeyError("Item: {} not in HashSet".format(item))

    def discard(self, item): # without the key error
        if HashSet.__remove(item, self.items): # check if we have removed item from the list self.items is True
            self.numItems -= 1

            load = max(self.numItems, 10) / len(self.items) # we'll not shrink list beyond 10
            
            if load <= 0.25:
                self.items = HashSet.__rehash(self.items, [None] * len(self.items)//2)

    # for item in HashSet #we'll overwrite the in with a magic function
    def __contains__(self, item): # We'll check if an item is in the HashSet - hash it, iterate over chain - return T/F. It's a magic function so it will be invoked auto. But it's not private function (it takes self), and can be called outside function.
        idx = hash(item) %  len(self.items)

        while self.items[idx] != None:
            if self.items[idx] == None:
                return None
            idx = (idx + 1) % len(self.items)
        return False

    # for item in HashSet (ignoring None and Placeholders)
    def __iter__(self):
        for i in range(len(self.items)):
            if (self.items[i] != None) and (type(self.items[i]) != HashSet.__Placeholder):
                yield self.items[i] # yield is like return, but it doesn't halt the for loop

    # HashSet A = {}

    def difference_update(self, other): # A = A - B / A.difference_update(B) -> A= = self, B = other
        for item in other: # based on the iter and contains
            self.discard(item)
        
    def difference(self, other): # C = A - B / C = A.difference(B) -> A = self, B = other
        result = HashSet(self)
        result.difference_update(other)
        return result

    def issuperset(self, other): # if every element of other contained in other
        for item in other:
            if item not in self:
                return False
        return True
    
    def clear(self): # delete all elements of cell
        self.numItems = 0
        self.items = [None] * 10

    def update(self, other):
        for item in other:
            self.add(item)

    # len(HashSet())
    def __len__(self):
        return self.numItems

    # For hashMap
    def __getitem__(self, item):
        idx = hash(item) %  len(self.item)
        while self.items[idx] != None:
            if self.items[idx] == item:
                return self.item[idx]
            idx = (idx + 1) % len(self.items)
        return None



#### Timo implementation

In [20]:
class HashSet:
    """A class to represent a set abstract data type."""   
    def __init__(self, contents=[]): # constructor is invoked automatically everytime the class is called for.
        """
        Constructs all the necessary attributes for the person object.
        Takes a list as a parameter & sets up items and numItems instance variables.
        """
        self.items = [None] * 10 # 10 empty element set
        self.numItems = 0 # intial content counter is 0

        # adds items form a given list to a items instance variable 
        for item in contents:
            self.add(item) # we'll define add function later - hashing, linear probing, rehashing... in modular parts
    
    class __Placeholder:
        """
        A class to represent a Placeholder type.
        Used for removing items that are not at the end of a chain.
        """
        def __init__(self):
            pass

        def __eq__(self, other):
            return False
        
    def __add(item, items):
        """
        Helper or private function responsible for:
        - calculating an index (hashing and %);
        - performing linear probing (collision resolution).
        It won't be used outside of the class.
        We don't need to call self in private function.
        """       
        # modulo (%) is required to keep hashes within list size range
        idx = hash(item) % len(items)

        loc = -1 # for checking collision cases

        # linear probing loop
        while items[idx] != None: # already element in idx index
            if items[idx] == item: # check duplicate
                return False
            
            if (loc < 0) and (type(items[idx]) == HashSet.__Placeholder): # if location is still not updated (-1), and the place is Placeholder
                loc = idx # store location

            idx = (idx + 1) % len(items) # new index within item length, continue loop till hits None or placeholder

        if loc < 0: # use the location received from while (straight None, linear probing None or placeholder)
            loc = idx
            
        items[loc] = item # use hash as index to store item

        return True

    def __rehash(oldList, newList):
        """
        Helper function responsible for rehashing values.
        Used when list size is changed due to a load factor reaching threshold.
        """ 
        for x in oldList:
            if (x != None) and (type(x) != HashSet.__Placeholder): # don't copy None or Placeholder
                HashSet.__add(x, newList) # take items of old list and add them to the new list
                
        return newList
    
    def __remove(item, items):
        """
        Helper function responsible for removing items from a chain.
        """       
        idx = hash(item) % len(items) # initial hash index of item

        # check for collision, loop to go through the items in the chain starting at hashed index
        while items[idx] != None: # if first hash value is not None
            if items[idx] == item: # if it's the item
                nextIdx = (idx + 1) % len(items) # find if next idx is None
                
                if items[nextIdx] == None:
                    items[idx] = None # substitute item with None if at the end of a chain
                
                else: # substitute item with Placeholder if not at the end of a chain
                    items[idx] = HashSet.__Placeholder()
                return True # replace item with None/Placeholder and return True

            idx = (idx + 1) % len(items)
        
        return False # if item doesn't exist, return False
    
    def __contains__(self, item):
        """
        Magic function responsible for checking if an item belongs to a set
        Invoked when "item in set" is executed.
        Overwrite global contains.
        Returns True if an item is in a set and False otherwise.
        Magic function so will be invoked auto, not private so takes self and can be called outside function.
        We'll check if an item is in the HashSet - hash it, iterate over chain - return T/F.
        """
        idx = hash(item) % len(self.items) # initial hash
        
        # loop to go through the items in the chain starting at hashed index. None and Placeholder is checked by magic __iter__ function.
        while self.items[idx] != None: # loop until end of set
            if self.items[idx] == item:
                return True # contains the item
            
            idx = (idx + 1) % len(self.items) # check next idx

        return False # if not in set

    def __iter__(self):
        """
        Magic function responsible for iterating over items in set
        Invoked when "for item in set" is executed.
        """
        for i in range(len(self.items)):
            # only yield items that are not None or Placeholders
            if (self.items[i] != None) and (type(self.items[i]) != HashSet.__Placeholder):
                yield self.items[i] # yield is like return, but it doesn't halt the for loop
                
    def __len__(self):
        """
        Magic function responsible for returning the length of a set
        Invoked when "len(set)" is executed
        numItems is a class attribute - item counter
        """
        return self.numItems
    
    def add(self, item):
        """
        Function responsible for adding items into a set.
        Doubles items list size when load factor >= 75%.
        """
        if HashSet.__add(item, self.items): # check if an unique element was added to the lsit. If it returns true, we check load factor
            self.numItems += 1 # update item counter
            load = self.numItems / len(self.items)
            # double items list size of load factor >= 75%
            if load >= 0.75:
                self.items = HashSet.__rehash(self.items, [None]*2*len(self.items))

    def remove(self, item):
        """
        Function responsible for removing items from a set.
        Halves items list size when load factor <= 25%.
        In addition, raises an exception when item is not in a set.
        """
        if HashSet.__remove(item, self.items): # if helper __remove returns True
            self.numItems -= 1
            load = max(self.numItems, 10) / len(self.items) # we won't shrink set beyond 10
            # halve items list size of load factor <= 25%
            if load <= 0.25:
                self.items = HashSet.__rehash(self.items, [None]*int(len(self.items)/2)) # take elements of old list, add them into fresh new list
        else: # __remove returned False as the value is not in the list
            raise KeyError("Item not in HashSet")

    def discard(self, item):
        """
        Function responsible for removing items from a set.
        Halves items list size when load factor <= 25%.
        Does not raise an exception when item is not in a set.
        """
        if HashSet.__remove(item, self.items):
            self.numItems -= 1
            load = max(self.numItems, 10) / len(self.items)
            if load <= 0.25:
                self.items = HashSet.__rehash(self.items, [None]*int(len(self.items)/2))
                
    def clear(self):
        """
        Function responsible for removing all elements of a set.
        Resets numItems instance variable to 0.
        Resets items instance variable with an empty list.
        """
        self.numItems = 0
        self.items = [None] * 10

    def update(self, other):
        """
        Function responsible for adding the items from one set to another set.
        """        
        for item in other:
            self.add(item)
            
    def difference_update(self, other):
        """
        Function responsible for subtracting from one set the elements of another set.
        A = A - B
        """  
        for item in other:
            self.discard(item)

    def difference(self, other):
        """
        Function responsible for subtracting from one set the elements of another set.
        C = A - B
        """
        result = HashSet(self) # C = A
        result.difference_update(other) # C = C - B
        
        return result

    def issuperset(self, other):
        """
        Function responsible for checking if one set is superset of another set
        Returns True if a set is a superset of another set and False otherwise.
        """
        for item in other:
            if item not in self: # not in invokes contians...
                return False
            
        return True
    
    def __getitem__(self, item):
        idx = hash(item) % len(self.items)
        while self.items[idx] != None:
            if self.items[idx] == item:
                return self.items[idx]
            idx = (idx + 1) % len(self.items)
        return None


In [4]:
# test before getting more complex
s = HashSet([1, 2, 2, 3, 4, 4, 5])

In [5]:
s.remove(2)

In [6]:
s.remove(2)

KeyError: 'Item not in HashSet'

### Sudoku

#### Class notes

In [ ]:
import sys
from HashSet import HashSet

def getColumn(matrix, colIndex): #we'll get 1 column and append to getGroup, then we'll get 1 square and append to getGroup
    col = []
    for rowIndex in range(9):
        col.append(matrix[rowIndex][colIndex])
    return col


def getSquare(matrix, rowIndex, colIndex): # 1 call of the function will just process 1 square (that's why +3 limit)
    square = [] # list for a single square where we'll add elements
    for i in range(rowIndex, rowIndex + 3): # range(0,3) -> [0, 1, 2] | range(0, 3) -> [0, 1, 2]
        for j in range(colIndex, colIndex + 3): # range(0,3) -> [0, 1, 2] | range(3, 6) -> [3, 4, 5]
            square.append(matrix[i][j]) # [0][0], [0][1], [0][2] | [1][0] # the starting point will be given by getGroups loop
    return square


def getGroups(matrix):
    groups = []

    for i in range(9):
        groups.append(len(matrix[i]))

    for i in range(9):
        groups.append(getColumn(matrix, i))

    for i in range(0, 9, 3): # range(0, 9, 3) -> [0, 3, 6 ]
        for j in range(0, 9, 3):
            groups.append(getSquares(matrix, i, j))
    
    return groups


def rule1(group):
    pass

def rule2(group):
    pass

def cardinality(x):
    return len(x) #HashSet function len on Set x

# we'll implement the two solution rules: reduction and ? before print and main
def reduceGroup(group):
    changed = False

    group.sort(key=cardinality)

    changed = rule2(group) # will return T/F
    changed = rule1(group) # will return T/F
    
    return changed # will return T/F, will be fed to reducedGroups, which'll be fed to reduce



def reduceGroups(groups): # take all groups
    changed = False
    for group in groups: # take 1 grouop at a time (row, col, sqr)
        if reduceGroup(group): # apply reduce rule, if it changes then change = True and continue loop
            changed = True
    return changed


def reduce(matrix):
    changed = True # assume changes by default
    groups = getGroups(matrix)

    while changed: # if change stays True, reduce groups. When reduceGroups returns False, break from loop.
        changed = reduceGroups(groups)

    # it doesn't return copy, it returns T/F for change in place, which'll be reflected in printMatrix


def printMatrix(matrix): # print that 
    for i in range(9):
        for j in range(9):
            if len(matrix[i][j]) != 1: # a cell is a HashSet, returns len of Hashset
                sys.stdout.write("x ")
            else:
                for k in matrix[i][j]:
                    sys.stdout.write(str(k) + " ")
        sys.stdout.write("\n")


def main():
    file = open("sudoku.txt", "r") #just read the file
    matrix = []

    for line in file:
        lst = line.split()
        row = []

        for val in lst:
            if val == 'x':
                s = HashSet(range(1,10))
            else:
                s = HashSet([eval(val)]) # take integer of val
            row.append(s)
        matrix.append(row)

    print("solving this puzzle:")
    print(matrix)

    reduce(matrix)
    print()
    print("Solution:")
    printMatrix(matrix)



main()


#### Timo implementation

In [ ]:
import sys # for file reading
from hashSet import HashSet

# We'll take 1 column (with 9 rows) and append to getGroup, then we'll get 1 square and append to getGroup
# 1 call 1 column
def getColumn(matrix, colIndex):
  col = [] # empty col
  for rowIndex in range(9):
    col.append(matrix[rowIndex][colIndex]) # 1-9 row for given column
    
  return col

# 1 call 1 square, that's why +3 limit
def getSquare(matrix, rowIndex, colIndex):
  square = []
  for i in range(rowIndex, rowIndex+3): # range(0,3) -> [0, 1, 2] | range(0, 3) -> [0, 1, 2]
    for j in range(colIndex,colIndex+3): # range(0,3) -> [0, 1, 2] | range(3, 6) -> [3, 4, 5]
        square.append(matrix[i][j]) # [0][0], [0][1], [0][2] | [1][0] # the starting point will be given by getGroups loop
        
  return square

def getGroups(matrix):
  groups = []
  # get rows
  for i in range(9):
    groups.append(list(matrix[i]))
  # get columns
  for i in range(9):
    groups.append(getColumn(matrix,i))
  # get squares
  # squares are processed left-right, up-down
  for i in range(0,9,3): # range(0, 9, 3) -> [0, 3, 6 ]
    for j in range(0,9,3):
      groups.append(getSquare(matrix,i,j))     

  return groups

def cardinality(x):
  return len(x)

def rule1(group):
  ### IMPLEMENT THIS FUNCTION ###

  changed = False
  
  # RULE 1 - You have to look for duplicate sets (i.e. set([1,6])). If you 
  # have same number of duplicate sets in a group (row, column, square) as 
  # the cardinality of the duplicate set size, then they must each get one 
  # value from the duplicate set. In this case the values of the duplicate 
  # set may be removed from all the other sets in the group. 

  
  # go through all the elements of the group which are alredy sorted from
  # smallest to largest cardinality

  # get the cardinality of the set       

  # if there are cardinality sets with cardinality elements then the other
  # sets can't have any of these values in them since these sets will have
  # to each have one of the cardinality values 

  # go through the sets and for each set different from the given set take
  # out all the elements that are in given set

  return changed
  
def rule2(group):
  ### IMPLEMENT THIS FUNCTION ###

  changed = False
  # RULE 2 - Reduce set size by throwing away elements that appear in other
  # sets in the group


  # pick an element of the group

  # for all the other elements of the group remove the elements that appear
  # in other elements of the group. These can be satisfied by other elements
  # of the group

  # When done, if there is one value left then it can only be satisfied by
  # this cell. This is a most constrained rule. If end up with 0 elements,
  # then not enough information yet to constrain this choice. If didn't
  # improve the situation at all, let's continue looking at other elements
  # in the row. 

  return changed

def reduceGroup(group): # takes cell?
  changed = False 
  # this sorts the sets from smallest to largest based cardinality
  group.sort(key=cardinality)
  changed = rule2(group) # will return T/F
  changed = rule1(group) # will return T/F
  
  return changed # will return T/F, will be fed to reducedGroups, which'll be fed to reduce

def reduceGroups(groups): # takes row, col, square
  changed = False
  for group in groups: # take 1 grouop at a time (row, col, sqr)
    if reduceGroup(group): # apply reduce rule, if it changes then change = True and continue loop
      changed = True
      
  return changed

def reduce(matrix):
    changed = True # assume changes by default
    groups = getGroups(matrix)
    
    while changed: # if change stays True, reduce groups (row, col, square). When reduceGroups returns False, break from loop.
        changed = reduceGroups(groups)
      # it doesn't return copy, it returns T/F for change in place, which'll be reflected in printMatrix

def printMatrix(matrix):
  for i in range(9):
    for j in range(9):
      if len(matrix[i][j]) != 1:
        sys.stdout.write("x ")
      else:
        for k in matrix[i][j]:
          sys.stdout.write(str(k) + " ")

    sys.stdout.write("\n")

def main():
  file = open(sys.argv[1], "r") #just read the file
  matrix = []

  for line in file:
    lst = line.split()
    row = []

    for val in lst:
      if val == 'x':
        s = HashSet(range(1,10))
      else:
        s = HashSet([eval(val)]) # or int(val)
      row.append(s)

    matrix.append(row)

  print("Solving this puzzle:")
  printMatrix(matrix)

  reduce(matrix)  

  print()
  print("Solution:")
  printMatrix(matrix)
  
main()


### HashMap: will be saved in a different folder

Class implementation

In [16]:
#from hashSet import HashSet

class HashMap:
    class __KVPair():
        def __init__(self, key, value):
            self.key = key
            self.value = value
        # =
        def __eq__(self, other): # invoked every time we'll use equal operator
            if type(self) != type(other):
                return False
            return self.key == other.key
        
        def getKey(self):
            return self.key
        
        def getValue(self):
            return self.value
        
        # hash(__KVPair)
        def __hash__(self): #we'll have to overwrite hashing for key-value pair
            return hash(self.key) #it'll hash key, not value

    def __init__(self): #constructor for hashmap, not for key-value pair
        self.hSet = HashSet()

    # len(HashMap()) - how many keys are there
    def __len__(self):
        return len(self.hSet) #type is HashSet, so it'll take appropriate len()
    
    # key in HashMap - use already defined contains for Key
    def __contains__(self, item):
        return HashMap.__KVPair(item, None) in self.hSet # first parameter is key, which is item here. Don't need value. The defined contains in HashSet took value, now it'll use the Hash function defined in KVPair, and Hash the Key.
    
    def not_contains(self, item):
        return item not in self.hSet
    
    # dict[key] = value
    def __setitem__(self, key, value):
        self.hSet.add(HashMap.__KVPair(key, value)) # hash key and store value against it

    # dict[key] -> value
    def __getitem__(self, key):
        if HashMap.__KVPair(key, None) in self.hSet:
            val = self.hSet[HashMap.__KVPair(key, None)].getValue()
            return val
        raise KeyError(f"Key {key} not in HashMap")
    
    # iterate over keys
    def __iter__(self):
        for kvpair in self.hSet:
            yield kvpair.getKey()
def main():
    d = HashMap()
    print(len(d))

    d["dog"] = "cat"
    d["batman"] = "joker"
    d["superman"] = "lex luther"

    for key in d:
        print(key)
    
    for key in d:
        print(key, d[key])

main()

0
superman
batman
dog


AttributeError: 'HashSet' object has no attribute 'item'

Timo implementation

In [21]:
class HashMap:

    class __KVPair:
        def __init__(self, key, value):
            self.key = key
            self.value = value

        def __eq__(self, other):
            if type(self) != type(other):
                return False
            return self.key == other.key

        def getKey(self):
            return self.key
        def getValue(self):
            return self.value

        def __hash__(self):
            return hash(self.key)

    def __init__(self):
        self.hSet = HashSet()

    def __len__(self):
        return len(self.hSet)

    def __contains__(self, item):
        return HashMap.__KVPair(item, None) in self.hSet
    def not__contains__(self, item):
        return item not in self.hSet

    def __setitem__(self, key, value):
        self.hSet.add(HashMap.__KVPair(key, value))

    def __getitem__(self, key):
        if HashMap.__KVPair(key, None) in self.hSet:
            val = self.hSet[HashMap.__KVPair(key, None)].getValue()
            return val

        raise KeyError(f"Key {key} not in HashMap")

    def __iter__(self):
        for x in self.hSet:
            yield x.getKey()

def main():
    d = HashMap()
    print(len(d))

    d["dog"] = "cat"
    d["batman"] = "joker"
    d["superman"] = "lex luther"

    for key in d:
        print(key)

    for key in d:
        print(key, d[key])

main()

0
superman
batman
dog
superman lex luther
batman joker
dog cat


Assignment: Memoization

In [22]:
# from hashMap import HashMap

# create HashMap to store calculated values

f = HashMap()

def fib(n):
    # if already calculated for n, then retrieve result from HashMap and return it
    if n in f:
        return f["n"]

    # if calculating for 0, then record result to HahsMap and return result
    elif n == 0:
      f["n"] = 0
      return 0

    # if calculating for 1, then record result to HahsMap and return result
    elif n == 1:
      f["n"] = 1
      return 1
    
    # if calculating for != 0, 1, value that is alredy in HashMap, then
    else:
        f["n"] = fib(n-1) + fib(n-2)
    
    # calculate value according to Fib formula, record the result to HashMap
    # and return the result

def main():
    print(fib(100))

main()


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

## Chapter 3: Trees

### Binary search trees

In [ ]:
class BinarySearchTree:
    class __Node:
        def __init__(self, val, left = None, right = None):
            self.val = val          # could be int, string
            self.left = left        # will be Node
            self.right = right      # will be Node

        def getVal(self):
            return self.val
        def setVal(self, newVal):
            self.val = newVal

        def getLeft(self):
            return self.left
        def getRight(self):
            return self.right
        
        def setLeft(self, newLeft):
            self.left = newLeft
        def setRight(self, newRight):
            self.right = newRight

        def __iter__(self):         # for something in Node - __iter__
            if self.left != None:
                for elem in self.left:  # self.left is also Node, it'll call __iter__ again
                    yield elem                
            yield self.val
            if self.right != None:
                for elem in self.right:  # self.right is also Node, it'll call __iter__ again
                    yield elem

    def __init__(self):
        # binary search only needs ref to root
        self.root = None    # initially
    
    def insert(self, val):
        self.root = BinarySearchTree.__insert(self.root, val)   #
    
    def __insert(root, val):
        # search until root pionts to None
        if root == None:
            return BinarySearchTree.__Node(val)  # if at point where root = None, create node where __Node(val, None, None)        
        if val < root.getVal():
            root.setLeft(BinarySearchTree.__insert(root.getLeft(), val)) # recursive call - instead of original root, use the left root        
        else:
            root.setRight(BinarySearchTree.__insert(root.getRight(), val))
        return root
    
    def delete(self, val):
        self.root = BinarySearchTree.__delete(self.root, val)

    def __delete(root, val):
        if root == None:
            return None # if after hitting root we find None, the val isn't in Node
        elif val < root.getVal():
            root.setLeft(BinarySearchTree.__delete(root.getLeft(), val))    # recurse delete with new root
        elif val > root.getVal():
            root.setRight(BinarySearchTree.__delete(root.getRight(), val))
        # Node that contains the value is root now
        else:
            if root.getLeft() == None:
                temp = root.getRight()
                root = None
                return temp # it'll be within setRight or setLeft, and will be the new val (either val or None)
            elif root.getRight() == None:
                temp = root.getLeft()
                root = None
                return temp
            temp = BinarySearchTree.__getInorderPredecessor(root.getLeft())   # get in order predecessor and overwrite val of where we're currently at
            root.setVal(temp.getVal())
            root.setLef(BinarySearchTree.__delete(root.getLeft(), temp.getVal()))

                
